In [ ]:
import gc
import torch
import sys
import json
import os
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import KFold
from google.colab import drive

# =============================================================================
# 0. Global Environment & Resource Management
# =============================================================================
print("📦 Initializing Environment...")

# Explicit memory cleanup to prevent OOM on Colab high-RAM instances
try:
    del model, tokenizer, optimizer, scheduler
except NameError:
    pass
torch.cuda.empty_cache()
gc.collect()

# Mount Google Drive for persistent storage of model checkpoints
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/"

# =============================================================================
# 1. Hyperparameters & Hardware Optimization (A100 Optimized)
# =============================================================================
CONFIG = {
    "model_name": "roberta-large", # Base model for fine-tuning

    # Dataset Paths
    "train_files": [
        os.path.join(BASE_PATH, "dev_track_a.jsonl"),
        os.path.join(BASE_PATH, "synthetic_data_for_contrastive_learning.jsonl")
    ],
    "test_file":  os.path.join(BASE_PATH, "test_track_a.jsonl"),

    # Output Configuration
    "output_file": os.path.join(BASE_PATH, "track_a_submission_roberta_5fold_fast_epoch10.jsonl"),

    # Training Parameters
    "max_len": 512,
    "batch_size": 32,      # High throughput for A100 GPU
    "accum_steps": 1,      # Gradient accumulation steps (1 for direct optimization)
    "epochs": 10,
    "lr": 1e-5,
    "n_folds": 5,          # Cross-validation folds
    "seed": 42,
    "device": "cuda",
    "num_workers": 4       # Multi-process data loading
}

def set_seed(seed):
    """Ensure reproducibility across experiments."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# =============================================================================
# 2. Data Engineering & Dataset Definition
# =============================================================================
def load_all_data(file_paths):
    """Load and parse JSONL files, supporting both ranking and contrastive formats."""
    all_data = []
    print("\n[Step 1] Loading Datasets...")
    for file_path in file_paths:
        if not os.path.exists(file_path):
            print(f"⚠️ Warning: {file_path} not found. Skipping.")
            continue
        print(f"📖 Reading: {os.path.basename(file_path)}")
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    item = json.loads(line)
                    # Standard ranking format
                    if item.get('anchor_text'):
                        all_data.append(item)
                    # Contrastive learning triplet format (Anchor, Pos, Neg)
                    elif item.get('anchor_story'):
                        anchor, pos, neg = item['anchor_story'], item['similar_story'], item['dissimilar_story']
                        # Randomly flip to prevent label bias
                        if random.random() > 0.5:
                            all_data.append({"anchor_text": anchor, "text_a": pos, "text_b": neg, "text_a_is_closer": True})
                        else:
                            all_data.append({"anchor_text": anchor, "text_a": neg, "text_b": pos, "text_a_is_closer": False})
                except Exception as e:
                    continue
    print(f"✅ Total Samples Loaded: {len(all_data)}")
    return np.array(all_data)

class RankingDataset(Dataset):
    """Custom Dataset for Pairwise Ranking tasks."""
    def __init__(self, data_list, tokenizer, max_len, is_test=False):
        self.data = data_list
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        anchor = item['anchor_text']

        # Tokenize both pairs: (Anchor, Text_A) and (Anchor, Text_B)
        pair_a = self.tokenizer(anchor, item['text_a'], truncation=True, max_length=self.max_len, padding="max_length", return_tensors="pt")
        pair_b = self.tokenizer(anchor, item['text_b'], truncation=True, max_length=self.max_len, padding="max_length", return_tensors="pt")

        inputs = {
            'input_ids_a': pair_a['input_ids'].squeeze(0),
            'mask_a': pair_a['attention_mask'].squeeze(0),
            'input_ids_b': pair_b['input_ids'].squeeze(0),
            'mask_b': pair_b['attention_mask'].squeeze(0),
        }

        if not self.is_test:
            # MarginRankingLoss Target: 1.0 if A is closer than B, else -1.0
            label = 1.0 if item.get('text_a_is_closer') else -1.0
            inputs['labels'] = torch.tensor(label, dtype=torch.float)

        return inputs

# =============================================================================
# 3. Training Pipeline (with Automated Mixed Precision - AMP)
# =============================================================================
def train_one_fold(fold_idx, train_data, val_data):
    """Executes the training loop for a single cross-validation fold."""
    print(f"\n🚀 Commencing Fold {fold_idx+1}/{CONFIG['n_folds']}")

    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
    model = AutoModelForSequenceClassification.from_pretrained(CONFIG['model_name'], num_labels=1).to(CONFIG['device'])

    train_ds = RankingDataset(train_data, tokenizer, CONFIG['max_len'])
    val_ds = RankingDataset(val_data, tokenizer, CONFIG['max_len'])

    # Optimized DataLoaders with pin_memory for faster host-to-device transfer
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=CONFIG['num_workers'], pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], num_workers=CONFIG['num_workers'], pin_memory=True)

    optimizer = AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=0.01)
    criterion = torch.nn.MarginRankingLoss(margin=0.5)

    # Initialize Gradient Scaler for AMP
    scaler = torch.cuda.amp.GradScaler()

    best_val_acc = 0
    save_path = os.path.join(BASE_PATH, f"roberta_fold_{fold_idx}.pt")

    for epoch in range(CONFIG['epochs']):
        model.train()
        total_loss = 0
        progress = tqdm(train_loader, desc=f"Fold {fold_idx+1} Epoch {epoch+1}", leave=False)

        for batch in progress:
            ids_a, mask_a = batch['input_ids_a'].to(CONFIG['device']), batch['mask_a'].to(CONFIG['device'])
            ids_b, mask_b = batch['input_ids_b'].to(CONFIG['device']), batch['mask_b'].to(CONFIG['device'])
            target = batch['labels'].to(CONFIG['device'])

            optimizer.zero_grad()

            # Forward pass with Autocast for FP16 precision
            with torch.cuda.amp.autocast():
                out_a = model(input_ids=ids_a, attention_mask=mask_a).logits.squeeze(-1)
                out_b = model(input_ids=ids_b, attention_mask=mask_b).logits.squeeze(-1)
                loss = criterion(out_a, out_b, target)

            # Backward pass with Scaled Gradients
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        # Validation phase
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in val_loader:
                ids_a, mask_a = batch['input_ids_a'].to(CONFIG['device']), batch['mask_a'].to(CONFIG['device'])
                ids_b, mask_b = batch['input_ids_b'].to(CONFIG['device']), batch['mask_b'].to(CONFIG['device'])
                target = batch['labels'].to(CONFIG['device'])

                with torch.cuda.amp.autocast():
                    s_a = model(input_ids=ids_a, attention_mask=mask_a).logits.squeeze(-1)
                    s_b = model(input_ids=ids_b, attention_mask=mask_b).logits.squeeze(-1)

                pred_a_better = s_a > s_b
                true_a_better = target > 0
                correct += (pred_a_better == true_a_better).sum().item()
                total += len(target)

        acc = correct / total
        print(f"   [Fold {fold_idx+1} | Epoch {epoch+1}] Validation Accuracy: {acc:.4f}")

        # Model Checkpointing
        if acc > best_val_acc:
            best_val_acc = acc
            torch.save(model.state_dict(), save_path)
            print(f"   🌟 New Best Accuracy. Model saved to {os.path.basename(save_path)}")

    # Memory cleanup after fold completion
    del model, optimizer, scaler, train_loader
    torch.cuda.empty_cache()
    gc.collect()

# =============================================================================
# 4. Inference & Model Ensembling
# =============================================================================
def inference_ensemble():
    """Performs inference using a 5-fold ensemble approach for maximum robustness."""
    print("\n[Step 3] Ensemble Inference (5-Fold Strategy)...")

    test_data_raw = []
    with open(CONFIG['test_file'], 'r') as f:
        for line in f:
            test_data_raw.append(json.loads(line))

    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
    test_ds = RankingDataset(test_data_raw, tokenizer, CONFIG['max_len'], is_test=True)
    test_loader = DataLoader(test_ds, batch_size=64, num_workers=CONFIG['num_workers'], pin_memory=True)

    all_fold_scores = np.zeros((len(test_data_raw), CONFIG['n_folds']))

    for fold in range(CONFIG['n_folds']):
        model_path = os.path.join(BASE_PATH, f"roberta_fold_{fold}.pt")
        if not os.path.exists(model_path):
            print(f"⚠️ Fold {fold} checkpoint not found. Skipping inference.")
            continue

        print(f"🤖 Inferencing with Fold {fold} weights...")
        model = AutoModelForSequenceClassification.from_pretrained(CONFIG['model_name'], num_labels=1).to(CONFIG['device'])
        model.load_state_dict(torch.load(model_path))
        model.eval()

        fold_preds = []
        with torch.no_grad():
            for batch in tqdm(test_loader, desc=f"Predicting Fold {fold}"):
                ids_a, mask_a = batch['input_ids_a'].to(CONFIG['device']), batch['mask_a'].to(CONFIG['device'])
                ids_b, mask_b = batch['input_ids_b'].to(CONFIG['device']), batch['mask_b'].to(CONFIG['device'])

                with torch.cuda.amp.autocast():
                    s_a = model(input_ids=ids_a, attention_mask=mask_a).logits.squeeze(-1)
                    s_b = model(input_ids=ids_b, attention_mask=mask_b).logits.squeeze(-1)
                    diff = s_a - s_b # Relative score difference
                fold_preds.extend(diff.cpu().numpy())

        all_fold_scores[:, fold] = fold_preds
        del model
        torch.cuda.empty_cache()

    # Averaging scores across all folds (Soft Voting/Ensemble)
    avg_scores = np.mean(all_fold_scores, axis=1)
    final_results = [{"text_a_is_closer": bool(score > 0)} for score in avg_scores]

    # Writing results to JSONL submission format
    with open(CONFIG['output_file'], 'w') as f:
        for res in final_results:
            f.write(json.dumps(res) + "\n")

    print(f"\n✅ Pipeline Complete! Submission file generated at: {CONFIG['output_file']}")

# =============================================================================
# 5. Main Execution Entry Point
# =============================================================================
if __name__ == "__main__":
    # Load and shuffle training data
    all_data = load_all_data(CONFIG['train_files'])

    # Define K-Fold Cross-Validation strategy
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])

    print(f"🔄 Starting {CONFIG['n_folds']}-Fold Cross Validation Workflow...")
    for fold, (train_idx, val_idx) in enumerate(kf.split(all_data)):
        train_fold = all_data[train_idx]
        val_fold = all_data[val_idx]
        train_one_fold(fold, train_fold, val_fold)

    # Run ensemble inference on test set
    inference_ensemble()